In [1]:
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.tree import plot_tree

In [2]:
raw_data      = pd.read_csv('../data/sentence-relations/train.csv', index_col='id')
raw_submissions = pd.read_csv('../data/sentence-relations/test.csv', index_col='id')

# Drop chinese and thai
raw_data = raw_data[raw_data['lang_abv'] != 'zh']
raw_data = raw_data[raw_data['lang_abv'] != 'th']

training_data, test_data = train_test_split(raw_data, test_size=0.2, random_state=42)

training_data.head()

,premise,hypothesis,lang_abv,language,label
id,,,,,
4e5ad9e03a,اب اتنا خفیہ تھا یہ.,یہ عوامی معلومات تھی۔,ur,Urdu,2
fea0d3c7e8,oh that's accommodating,That is disruptive.,en,English,2
4ce586b61e,more than anything else in this day and age th...,"In your decisions age is a big factor, and I a...",en,English,1
0dbcd1012b,"Aie! les boucaniers en-dessous s'écriaient, et...",Les Buccaneers étaient bruyants quand ils disa...,fr,French,0
157895e59c,"一个采购战略组织，是壮大的力资本发展战略的一部分, 这是在原则VI上讨论的。",原则四涉及财富500强机构的资本发展战略。,zh,Chinese,1


In [3]:
from transformers import DataCollatorWithPadding
from transformers import AutoTokenizer

roberta_tokenizer = AutoTokenizer.from_pretrained("roberta-base", truncation=True)
data_collator = DataCollatorWithPadding(tokenizer=roberta_tokenizer)

/home/alex/Documents/Studium/2-Master/ki-lab-ss23/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/alex/Documents/Studium/2-Master/ki-lab-ss23/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
from torch.utils.data import Dataset, DataLoader

class BertSentenceDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer

        self.X = self.data['premise'] + '[SEP]' + self.data['hypothesis']
        self.y = self.data['label']

        self.encoded = self.tokenizer(self.X.tolist(), padding=True, truncation=True)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return {
            'input_ids': self.encoded['input_ids'][idx],
            'attention_mask': self.encoded['attention_mask'][idx],
            'label': self.y.iloc[idx]
        }

bert_train_dataset = BertSentenceDataset(training_data, roberta_tokenizer)
bert_test_dataset = BertSentenceDataset(test_data, roberta_tokenizer)

In [5]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

id2label = {0: "entailment", 1: "neutral", 2: "contradiction"}
label2id = {"entailment": 0, "neutral": 1, "contradiction": 2}
num_labels = len(id2label)

model = AutoModelForSequenceClassification.from_pretrained("roberta-base", num_labels=num_labels, id2label=id2label, label2id=label2id)

/home/alex/Documents/Studium/2-Master/ki-lab-ss23/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

training_args = TrainingArguments(
    output_dir="roberta-base-sentence-relation",
    learning_rate=1e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=10,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="macro")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

In [7]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=bert_train_dataset,
    eval_dataset=bert_test_dataset,
    tokenizer=roberta_tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

  2%|▏         | 500/24240 [02:13<1:46:03,  3.73it/s]

{'loss': 1.1054, 'grad_norm': 4.7216796875, 'learning_rate': 9.793729372937294e-06, 'epoch': 0.21}


  4%|▍         | 1000/24240 [04:27<1:44:33,  3.70it/s]

{'loss': 1.1033, 'grad_norm': 7.050784587860107, 'learning_rate': 9.587458745874588e-06, 'epoch': 0.41}


  6%|▌         | 1500/24240 [06:42<1:41:47,  3.72it/s]

...

{'eval_loss': 1.7713961601257324, 'eval_accuracy': 0.6646039603960396, 'eval_f1': 0.6622007920893216, 'eval_precision': 0.6637151931563942, 'eval_recall': 0.6623837111198605, 'eval_runtime': 47.6334, 'eval_samples_per_second': 50.889, 'eval_steps_per_second': 12.722, 'epoch': 9.0}


 91%|█████████ | 22000/24240 [1:46:13<09:59,  3.74it/s]   

{'loss': 0.4291, 'grad_norm': 0.01362778153270483, 'learning_rate': 9.240924092409241e-07, 'epoch': 9.08}


 93%|█████████▎| 22475/24240 [1:48:21<07:54,  3.72it/s]

KeyboardInterrupt: 

In [ ]:
# Get the best model
model = AutoModelForSequenceClassification.from_pretrained('roberta-base-sentence-relation/checkpoint-4816', num_labels=num_labels, id2label=id2label, label2id=label2id)

# Evaluate the model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=bert_train_dataset,
    eval_dataset=bert_test_dataset,
    tokenizer=None,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.evaluate()

100%|██████████| 86/86 [00:08<00:00, 10.50it/s]


{'eval_loss': 1.6948167085647583,
 'eval_accuracy': 0.7991266375545851,
 'eval_f1': 0.7975053326129699,
 'eval_precision': 0.7997977316986625,
 'eval_recall': 0.7971155023486721,
 'eval_runtime': 8.2844,
 'eval_samples_per_second': 165.854,
 'eval_steps_per_second': 10.381}